In [207]:
import copy
import time
import random
import itertools
import numpy as np
import networkx as nx

In [194]:
random_fitness = lambda: random.uniform(-10, 10)

class Node(object):
    def __init__(self, name, value=None):
        self.value = value
        self.name = name
    
    def __str__(self):
        return str(self.name)

In [227]:
values = [0, 1] # binary strings
colors = {0: 'b', 1: 'r'}

class Model(object):
    def __init__(self, n, k, graph=None, fitness_table=None):
        self.n, self.k = n, k
        self.graph = graph or self.generate_graph()
        self.fitness_table = fitness_table or self.random_fitness()
        self.nodes = list(self.graph.nodes)
    
    def generate_graph(self):
        nodes = [Node(i, random.choice(values))
                 for i in range(self.n)]
        g = nx.Graph()
        for i in range(self.n):
            for j in range(1, self.k+1):
                g.add_edge(nodes[i], nodes[(i+j) % self.n])
        return g

    def random_fitness(self):
        xs = itertools.product(*([values] * (self.k+1)))
        return {x: random_fitness() for x in xs}
    
    def fitness(self):
        total_fitness = 0
        node_vals = [n.value for n in self.nodes]
        for i in range(self.n):
            j = (i + self.k)
            lookup = np.take(node_vals, range(i, j+1), mode='wrap')
            total_fitness +=  self.fitness_table[tuple(lookup)]
        return total_fitness
    
    def flip_random_bit(self):
        node = random.choice(self.nodes)
        node.value = random.choice(values)
    
    def draw(self, ax):
        cs = [colors[n.value] for n in self.nodes]
        nx.draw_shell(model.graph, node_color=cs, with_labels=True)

In [228]:
def optimize(model, niters=100):
    for i in range(niters):
        prev_model = copy.deepcopy(model)
        prev_fit = prev_model.fitness()
        model.flip_random_bit()
        new_fit = model.fitness()
        yield model
        
        if new_fit < prev_fit:
            model = prev_model

In [229]:
model = Model(5, 1)
print(model.fitness_table.keys())
print([n.value for n in model.nodes])
model.fitness()

dict_keys([(0, 0), (0, 1), (1, 0), (1, 1)])
[1, 0, 1, 1, 1]


5.810623048405285

In [240]:
%matplotlib inline
from IPython import display
from IPython.html import widgets
from IPython.display import display, clear_output

from matplotlib import pyplot as plt

In [ ]:
f = plt.gcf()
ax = plt.gca()
out = widgets.Output()

for model in optimize(model):
    with out:
        model.draw(ax)
        clear_output(wait=True)
        display(f)
    time.sleep(0.2)